The goal of this workbook is to use APIs to get the following:
- Daily open, high, low, close, adjusted close, volume, dividend amount, and split coefficient
- Google trends score

For the period 01.02.18-06.1.21

Step one will be to create a function that returns the data for one ticker. Then I'll use that function to get the data for all of my tickers

## Collecting Stock Data Using Alpha Vantage

In [4]:
key = '49KFJCFS3P6CEU4F'

In [3]:
from alpha_vantage.timeseries import TimeSeries

In [38]:
#time series daily adjusted. Works but only goes back to 9/19
import requests

# replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
url = 'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol=IBM&apikey=49KFJCFS3P6CEU4F&outputsize=full'
r = requests.get(url)
data = r.json()

print(data['Time Series (Daily)']['2021-06-01'])
print(data['Time Series (Daily)']['2018-01-02'])

{'1. open': '145.0', '2. high': '145.83', '3. low': '143.75', '4. close': '144.19', '5. adjusted close': '144.19', '6. volume': '2417455', '7. dividend amount': '0.0000', '8. split coefficient': '1.0'}
{'1. open': '154.5', '2. high': '154.81', '3. low': '153.54', '4. close': '154.25', '5. adjusted close': '130.577642951', '6. volume': '4195225', '7. dividend amount': '0.0000', '8. split coefficient': '1.0'}


The alpha vantage api is able to get the data for the time frame we want. Next steps of creating the stock data collection function:
- translate the json data into a dataframe
- account for stocks that had IPOs after 2018-01-02

# Translate JSON to DataFrame


## Collecting trends data using Google Trends API

In [40]:
import pandas as pd                        
from pytrends.request import TrendReq
pytrend = TrendReq()

In [53]:
pytrend.build_payload(kw_list=['IBM'])
# Interest by Region
df = pytrend.interest_over_time()
df

,IBM,isPartial
date,,
2016-07-10,75,False
2016-07-17,82,False
2016-07-24,77,False
2016-07-31,72,False
2016-08-07,75,False
...,...,...
2021-06-06,51,False
2021-06-13,48,False
2021-06-20,49,False
